In [ ]:
! pip install wandb

In [69]:
import tensorflow as tf
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
import numpy as np
import cv2
import wandb
from wandb.keras import WandbCallback
import matplotlib.pyplot as plt

In [92]:
wandb.init(project="cifar100")

accuracy,▃▄▄▅▅▅▅▅▅▆▁▁▁▁▁▁▁▁▁▁▄▅▆▆▆▆▇▇▇▇▇▇▇███████
epoch,▁▂▃▃▄▅▆▆▇█▁▂▃▃▄▅▆▆▇█▁▂▃▃▄▅▆▆▇█▁▂▃▃▄▅▆▆▇█
loss,▆▅▅▅▄▄▄▄▄▄██████████▅▄▄▃▃▃▃▂▂▂▂▂▂▂▁▁▁▁▁▁
val_accuracy,▅▅▆▆▆▇▆▇▇▇▁▁▁▁▁▂▂▂▂▂▆▇▇█████████████████
val_loss,▄▄▃▃▃▃▃▂▂▂██████████▃▂▂▁▁▁▁▁▁▁▂▂▂▂▂▃▃▃▄▄
accuracy,0.92396
best_epoch,6
best_val_loss,0.86179
epoch,9
loss,0.2098
val_accuracy,0.6884


## **Dataset**

In [93]:
cifar100 = tf.keras.datasets.cifar100

(X_train, Y_train), (X_rem, Y_rem) = cifar100.load_data()
X_val, X_test, Y_val, Y_test = train_test_split(X_rem,Y_rem,shuffle=True, test_size=0.5)

#normalization
X_train = X_train / 255.0
X_val = X_val / 255.0

169017344/169001437 [==============================] - 11s 0us/step


**Model**

In [94]:
model = tf.keras.models.Sequential([
    # استخراج ویژگی
    layers.Conv2D(32, (3,3), activation='relu', input_shape=(32,32,3)),
    layers.MaxPooling2D(),
    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(64, (5,5), activation='relu'),

    layers.Flatten(),

    #همون شبکه ساده قدیمی
    layers.Dense(128, activation='relu'),
    layers.Dense(100, activation='softmax')
])

In [ ]:
model.summary()

In [95]:
config = wandb.config
config.learning_rate = 0.001

In [96]:
model.compile(optimizer=tf.keras.optimizers.Adam(config.learning_rate),
              loss = tf.keras.losses.sparse_categorical_crossentropy,
              metrics=['accuracy']) 

In [97]:
model.fit(X_train, Y_train, epochs=10 , validation_data=(X_val, Y_val),callbacks=[WandbCallback()])

Epoch 1/10
1563/1563 [==============================] - 13s 8ms/step - loss: 3.9263 - accuracy: 0.0942 - val_loss: 3.5285 - val_accuracy: 0.1536 - _timestamp: 1646832384.0000 - _runtime: 86.0000
Epoch 2/10
1563/1563 [==============================] - 12s 8ms/step - loss: 3.3485 - accuracy: 0.1879 - val_loss: 3.1621 - val_accuracy: 0.2220 - _timestamp: 1646832397.0000 - _runtime: 99.0000
Epoch 3/10
1563/1563 [==============================] - 12s 8ms/step - loss: 3.0578 - accuracy: 0.2407 - val_loss: 2.9595 - val_accuracy: 0.2662 - _timestamp: 1646832409.0000 - _runtime: 111.0000
Epoch 4/10
1563/1563 [==============================] - 12s 8ms/step - loss: 2.8577 - accuracy: 0.2799 - val_loss: 2.8534 - val_accuracy: 0.2858 - _timestamp: 1646832421.0000 - _runtime: 123.0000
Epoch 5/10
1563/1563 [==============================] - 13s 8ms/step - loss: 2.7018 - accuracy: 0.3121 - val_loss: 2.8183 - val_accuracy: 0.2908 - _timestamp: 1646832434.0000 - _runtime: 136.0000
Epoch 6/10
1563/1563 [

In [98]:
model.save("cifar100.h5")

In [99]:
model.evaluate(X_test,Y_test)

157/157 [==============================] - 1s 5ms/step - loss: 632.5463 - accuracy: 0.1288


[632.5463256835938, 0.12880000472068787]

In [ ]:
img = cv2.imread("3.png")
img = cv2.resize(img, (28,28))

img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

img = img / 255.0
img = img.reshape(1,28,28)

result = model.predict([img])
np.argmax(result)